### ### SETUP

In [6]:
%pip install langchain-ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gemma3:4b",
    temperature=0
)


EVALUATOR_LLM = LangchainLLMWrapper(llm)

C:\Users\user\AppData\Local\Temp\ipykernel_15468\3772450948.py:9: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  EVALUATOR_LLM = LangchainLLMWrapper(llm)


### WRITE TEST TO CHECK MY LOCAL LLM RESPONSE


In [10]:

from ragas.llms import LangchainLLMWrapper
from ragas import evaluate
from ragas.metrics import LLMContextRecall
from datasets import Dataset


data = {
    "question": [
        "Who is the current president of the United States?"
    ],
    "answer": [
        "Joe Biden"
    ],
    "contexts": [[
        "Joe Biden serves as the current president of America"
    ]],
    "ground_truth": [
        "Joe Biden serves as the current president of America"
    ]
}

dataset = Dataset.from_dict(data)

result = evaluate(
    dataset=dataset,
    metrics=[LLMContextRecall()],
    llm=EVALUATOR_LLM,
)
result


C:\Users\user\AppData\Local\Temp\ipykernel_15468\2515294875.py:3: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

{'context_recall': 1.0000}

### USE SAME DATA FOR CONTEXT PRECISION


In [ ]:
from ragas.metrics import ContextPrecision
data = {
    "question": [
        "Who is the current president of the United States?"
    ],
    "answer": [
        "Joe Biden"
    ],
    "contexts": [[
        "Joe Biden serves as the current president of the United States.",
        "The Eiffel Tower is in Paris.",
        "Bananas are berries.",
        "The moon has no atmosphere."
    ]],
    "ground_truth": [
        "Joe Biden"
    ]
}

dataset = Dataset.from_dict(data)
context_relevancy = ContextPrecision(llm=EVALUATOR_LLM)

result = evaluate(
    dataset=dataset,
    metrics=[context_relevancy],
)

result.to_pandas()


C:\Users\user\AppData\Local\Temp\ipykernel_5024\3973006881.py:1: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import ContextPrecision


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision
0,Who is the current president of the United Sta...,[Joe Biden serves as the current president of ...,Joe Biden,Joe Biden,1.0


In [13]:
from ragas.metrics import ContextPrecision
data = {
    "question": [
        "Who is the capital of Macedonia?"
    ],
    "answer": [
        "Skopje"
    ],
    "contexts": [[
    "Skopje is the capital city of Macedonia",
    "The Eiffel Tower is in Paris."
    ]],
    "ground_truth": [
        "Skopje"
    ]
}

dataset = Dataset.from_dict(data)
context_relevancy = ContextPrecision(llm=EVALUATOR_LLM)

result = evaluate(
    dataset=dataset,
    metrics=[context_relevancy],
)

result.to_pandas()


C:\Users\user\AppData\Local\Temp\ipykernel_15468\639935256.py:1: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import ContextPrecision


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision
0,Who is the capital of Macedonia?,"[Skopje is the capital city of Macedonia, The ...",Skopje,Skopje,1.0


### TEST CASE WHERE ANSWER COMES FROM LLM

### CREATE GENERATE ANSWER FUNCTION

In [8]:
def generate_answer(llm, question, contexts):
    prompt = build_prompt(question, contexts)
    response = llm.invoke(prompt)
    return response.content

def build_prompt(question, contexts):
    context_block = "\n".join(contexts)
    return f"""
Use the following context to answer the question.

Context:
{context_block}

Question:
{question}

Answer:
"""

def generate_answer(llm, question, contexts):
    prompt = build_prompt(question, contexts)
    response = llm.invoke(prompt)
    return response.content


### USE THE ANSWER TO THE TEST CASE

In [ ]:
from datasets import Dataset

question = "Who is the current president of the United States?"

contexts = [
    "Joe Biden serves as the current president of America"
]

answer = generate_answer(llm, question, contexts)

data = {
    "question": [question],
    "answer": [answer],            
    "contexts": [contexts],
    "ground_truth": [
        "Joe Biden serves as the current president of America"
    ]
}

dataset = Dataset.from_dict(data)

from ragas import evaluate
from ragas.metrics import ContextPrecision
from ragas.llms import LangchainLLMWrapper

EVALUATOR_LLM = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=dataset,
    metrics=[ContextPrecision(llm=EVALUATOR_LLM)],
)

result.to_pandas()



C:\Users\user\AppData\Local\Temp\ipykernel_15468\3476736495.py:23: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import ContextPrecision
C:\Users\user\AppData\Local\Temp\ipykernel_15468\3476736495.py:26: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  EVALUATOR_LLM = LangchainLLMWrapper(llm)


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision
0,Who is the current president of the United Sta...,[Joe Biden serves as the current president of ...,Joe Biden.,Joe Biden serves as the current president of A...,1.0
